In [1]:
import openpyxl as excel

In [2]:
template_file = '/Users/fall_f/Desktop/automation_app/xlsx file/invoice-template.xlsx'
save_file = 'invoice01.xlsx'

name = '田中一郎'
subject = '1月分のご請求'
items = [
    ['りんご', 5, 320],
    ['バナナ', 8, 210],
    ['メロン', 1, 1200]
]

In [3]:
book = excel.load_workbook(template_file)
sheet = book.active
sheet['B4'] = name
sheet['C10'] = subject
total =0
for i, it, in enumerate(items):
    summary, count, price = it
    subtotal = count * price
    total += subtotal
    row = 15 + i
    sheet.cell(row, 2,summary)
    sheet.cell(row, 5, count)
    sheet.cell(row, 6, price)
    sheet.cell(row, 7, subtotal)
sheet['C11'] = total
book.save('save_file.xlsx')

/opt/anaconda3/lib/python3.11/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 請求書!$A:$H.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


In [1]:
import glob
files = glob.glob('*.xlsx')
print(files)

['save_file.xlsx']


In [5]:
import glob
import openpyxl as excel

target_dir = './salesbooks'
save_file = 'matome.xlsx'

#ファイルの読み込み処理
def read_files():
    book = excel.Workbook()
    main_sheet = book.active
    enumfiles(main_sheet)
    book.save(save_file)

def enumfiles(main_sheet):
    files =glob.glob(target_dir + '/*.xlsx')
    for fname in files:
        read_book(main_sheet,fname)

def read_book(main_sheet, fname):
    print('read :'  + fname)
    book = excel.load_workbook(fname, data_only= True)
    sheet = book.active
    rows =sheet['A4' : 'F999']
    for row in rows:
        values = [cell.value for cell in row]
        if values[0] is None:
            break
        main_sheet.append(values)

if __name__ == '__main__':
    read_files()


read :./salesbooks/sales-tanaka.xlsx
read :./salesbooks/sales-kakei.xlsx
read :./salesbooks/sales-ooda.xlsx
read :./salesbooks/sales-yamda.xlsx
read :./salesbooks/sales-inoue.xlsx
